In [1]:
!pip install datasets

In [2]:
!pip install spacy
import spacy

!python -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')

     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/587.7 MB 307.8 kB/s eta 0:31:09
ERROR: Exception:
Traceback (most recent call last):
  File "/home/hari/anaconda3/lib/python3.9/site-packages/pip/_vendor/urllib3/response.py", line 435, in _error_catcher
    yield
  File "/home/hari/anaconda3/lib/python3.9/site-packages/pip/_vendor/urllib3/response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/home/hari/anaconda3/lib/python3.9/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "/home/hari/anaconda3/lib/python3.9/http/client.py", line 463, in read
    n = self.readinto(b)
  File "/home/hari/anaconda3/lib/python3.9/http/client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "/home/hari/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "/home/hari/anaconda3/lib/python3.9/ssl.py", line 1242, in recv_into
    return self.read(nby

In [3]:
from datasets import load_dataset

dataset = load_dataset('paws', 'labeled_final')

In [4]:
import pandas as pd

train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])
val = pd.DataFrame(dataset['validation'])

In [5]:
sentence1 = train.at[0, 'sentence1']
sentence2 = train.at[0, 'sentence2']

In [6]:
sent1 = nlp(sentence1)
sent2 = nlp(sentence2)

In [7]:
ents1 = [i.text for i in sent1.ents]

In [8]:
ents2 = [i.text for i in sent2.ents]

In [9]:
set(ents1) == set(ents2)

True

In [10]:
def entity_match(sentence1, sentence2):
  t1 = nlp(sentence1)
  t2 = nlp(sentence2)

  e1 = set([i.text for i in t1.ents])
  e2 = set([i.text for i in t2.ents])

  try:
    return len(e1.intersection(e2)) / max([len(e1), len(e2)])
  except:
    return None

In [11]:
entity_match(sentence1, sentence2)

1.0

In [12]:
def sentence_similarity(sentence1, sentence2):
  d1 = nlp(sentence1)
  d2 = nlp(sentence2)

  return d1.similarity(d2)

In [13]:
!python -m pip install git+https://github.com/mmxgn/spacy-clausie.git

import claucy

  Cloning https://github.com/mmxgn/spacy-clausie.git to /tmp/pip-req-build-kbal1wsp
  Running command git clone --filter=blob:none --quiet https://github.com/mmxgn/spacy-clausie.git /tmp/pip-req-build-kbal1wsp
  Resolved https://github.com/mmxgn/spacy-clausie.git to commit ac8c7d9ee89f3ed26344e4a0ce4f58ef63df3a87
  Preparing metadata (setup.py) ... done


In [14]:
import spacy
import claucy

nlp = spacy.load('en_core_web_lg')
claucy.add_to_pipe(nlp)

doc = nlp(train.at[3, 'sentence1'])

props = doc._.clauses[0].to_propositions(as_text=True)

In [15]:
props

['comparable rates be maintained When']

In [16]:
for clause in doc._.clauses:
  print(clause.to_propositions(as_text=True))

['comparable rates be maintained When']
['the results are high']


In [17]:
d2 = nlp(train.at[3, 'sentence2'])

for p in d2._.clauses:
  print(p.to_propositions(as_text=True))


['The results are high']
['comparable flow rates be maintained when']


In [18]:
t = nlp('he asked him for a passport to returned to England through Scotland')
t2 = nlp('he asked him for a passport to returned to Scotland through England')

t.similarity(t2)

1.000000001914347

In [19]:
def clause_similarity(sentence1, sentence2):
  d1 = nlp(sentence1)
  d2 = nlp(sentence2)

  try:
    cl1 = []
    for c in d1._.clauses:
      cl1.extend(c.to_propositions(as_text=True))

    cl2 = []
    for c in d2._.clauses:
      cl2.extend(c.to_propositions(as_text=True))

    avg_similarity = 1
    for clause1 in cl1:
      max_similarity = 0
      for clause2 in cl2:
        cnlp1 = nlp(clause1)
        cnlp2 = nlp(clause2)
        sim = cnlp1.similarity(cnlp2)
        if sim > max_similarity:
          max_similarity = sim
      avg_similarity *= max_similarity

    return avg_similarity
  except:
    return None



In [20]:
clause_similarity(sentence1, sentence2)

0.6671183047318059

In [21]:
clause_similarity(train.at[3, 'sentence1'], train.at[3, 'sentence2'])

0.9049447687149189

In [25]:
train_copy = train.head(7000).copy()
test_copy = test.head(1000).copy()
val_copy = val.head(2000).copy()

In [28]:
train_copy['entity_match'] = train_copy.apply(lambda x: entity_match(x['sentence1'], x['sentence2']), axis=1)
train_copy['sentence_similarity'] = train_copy.apply(lambda x: sentence_similarity(x['sentence1'], x['sentence2']), axis=1)

train_copy['clause_similarity'] = train_copy.apply(lambda x: clause_similarity(x['sentence1'], x['sentence2']), axis=1)

test_copy['entity_match'] = test_copy.apply(lambda x: entity_match(x['sentence1'], x['sentence2']), axis=1)
test_copy['sentence_similarity'] = test_copy.apply(lambda x: sentence_similarity(x['sentence1'], x['sentence2']), axis=1)

test_copy['clause_similarity'] = test_copy.apply(lambda x: clause_similarity(x['sentence1'], x['sentence2']), axis=1)

val_copy['entity_match'] = val_copy.apply(lambda x: entity_match(x['sentence1'], x['sentence2']), axis=1)
val_copy['sentence_similarity'] = val_copy.apply(lambda x: sentence_similarity(x['sentence1'], x['sentence2']), axis=1)

val_copy['clause_similarity'] = val_copy.apply(lambda x: clause_similarity(x['sentence1'], x['sentence2']), axis=1)


/tmp/ipykernel_77467/2746356500.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = cnlp1.similarity(cnlp2)


In [29]:
train_copy = train_copy.dropna()

test_copy = test_copy.dropna()

val_copy = val_copy.dropna()

In [30]:
X_train = train_copy.loc[:, ['entity_match', 'sentence_similarity', 'clause_similarity']].values
y_train = train_copy['label'].values

X_test = test_copy.loc[:, ['entity_match', 'sentence_similarity', 'clause_similarity']].values
y_test = test_copy['label'].values

X_val = val_copy.loc[:, ['entity_match', 'sentence_similarity', 'clause_similarity']].values
y_val = val_copy['label'].values

In [44]:
import numpy as np

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score


md = SVC()

md.fit(np.array(X_train).astype(np.float64), np.array(y_train).astype(np.float64))
y_pred = md.predict(np.array(X_test).astype(np.float64))


confusion_matrix(y_pred, y_test)

array([[350, 263],
       [ 88,  97]])

In [45]:
accuracy_score(y_pred, y_test)

0.5601503759398496

In [34]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score


md = LogisticRegression()

md.fit(np.array(X_train).astype(np.float64), np.array(y_train).astype(np.float64))
y_pred = md.predict(np.array(X_test).astype(np.float64))


confusion_matrix(y_pred, y_test)

array([[438, 360],
       [  0,   0]])

In [35]:
X_train.shape

(5989, 3)

In [36]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

2024-02-15 17:06:54.764322: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-15 17:06:54.766810: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 17:06:54.785515: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 17:06:54.785540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 17:06:54.786224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Epoch 1/100
94/94 [==============================] - 1s 3ms/step - loss: 0.6868 - accuracy: 0.5564 - val_loss: 0.6849 - val_accuracy: 0.5629
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6852 - accuracy: 0.5575 - val_loss: 0.6905 - val_accuracy: 0.5403
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6846 - accuracy: 0.5535 - val_loss: 0.6833 - val_accuracy: 0.5617
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6846 - accuracy: 0.5605 - val_loss: 0.6823 - val_accuracy: 0.5629
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6850 - accuracy: 0.5527 - val_loss: 0.6821 - val_accuracy: 0.5629
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6834 - accuracy: 0.5585 - val_loss: 0.6822 - val_accuracy: 0.5629
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6839 - accuracy: 0.5557 - val_loss: 0.6825 - val_accuracy: 0.5629
Epoch 8/100
9

Epoch 59/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6797 - accuracy: 0.5664 - val_loss: 0.6800 - val_accuracy: 0.5762
Epoch 60/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6798 - accuracy: 0.5704 - val_loss: 0.6798 - val_accuracy: 0.5768
Epoch 61/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6793 - accuracy: 0.5707 - val_loss: 0.6799 - val_accuracy: 0.5786
Epoch 62/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6790 - accuracy: 0.5694 - val_loss: 0.6814 - val_accuracy: 0.5768
Epoch 63/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6790 - accuracy: 0.5762 - val_loss: 0.6794 - val_accuracy: 0.5803
Epoch 64/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6793 - accuracy: 0.5665 - val_loss: 0.6799 - val_accuracy: 0.5820
Epoch 65/100
94/94 [==============================] - 0s 2ms/step - loss: 0.6790 - accuracy: 0.5652 - val_loss: 0.6816 - val_accuracy: 0.5739
Epoch 

In [37]:
y_pred = model.predict(X_test)

y_pred = [1 if i>0.5 else 0 for [i] in y_pred]

25/25 [==============================] - 0s 1ms/step


In [38]:
confusion_matrix(y_pred, y_test)

array([[350, 264],
       [ 88,  96]])

In [39]:
accuracy_score(y_pred, y_test)

0.5588972431077694

,id,sentence1,sentence2,label
1,2,His father emigrated to Missouri in 1868 but r...,"His father emigrated to America in 1868 , but ...",0
3,4,"Steiner argued that , in the right circumstanc...",Steiner held that the spiritual world can be r...,0
4,5,"Luciano Williames Dias ( born July 25 , 1970 )...",Luciano Williames Dias ( born 25 July 1970 ) i...,0
5,6,"During her sophomore , junior and senior summe...","During her second , junior and senior summers ...",1
6,7,The smallest number that can be represented in...,The smallest number that can be represented as...,0
...,...,...,...,...
995,996,Warren Zevon wrote the song `` The Envoy '' fr...,"Habib wrote the song `` The Envoy '' , from hi...",0
996,997,"According to the United States Census Bureau ,...","According to the United States Census Bureau ,...",1
997,998,"Afterwards , he added his own surname to Bhosl...",He always added his own surname of Bhosle and ...,0
998,999,"If they were killed , Bibi would be the first ...","If killed , Bibi would be the first woman in P...",1


In [48]:
output_sentences = test.loc[test_copy.index, :].copy()

output_sentences['pred'] = md.predict(X_test)

In [49]:
output_sentences.to_csv('output_sentences.csv')